In this final project we \color{orange} TODO \color{black}

Import the libraries

In [1]:
path = "../Data/DataTrain"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import modin.pandas as mpd # Use optimized pandas wrapper for multi-threading processing?

import chess
import torch

Use GPU \color{orange} TODO \color{black}

In [3]:
print(torch.cuda.is_available() ) # Is there a graphics card?
print(torch.cuda.device_count() ) # How many devices can use cuda?
# print(torch.cuda.current_device() ) # Which device is currently being used?
# print(torch.cuda.device(0) )
# print(torch.cuda.get_device_name(0) ) # What is the name of the device being used?

True
1


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Cite the dataset and read it in \color{orange} TODO \color{black}

In [6]:
df = pd.read_csv(f'{path}/Chess_Jan_aa', engine = 'pyarrow')
# df = mpd.read_csv('../Data/lichess_db_standard_rated_2019-01.csv')
print(list(df.columns))

['game_id', 'type', 'result', 'white_player', 'black_player', 'white_elo', 'black_elo', 'time_control', 'num_ply', 'termination', 'white_won', 'black_won', 'no_winner', 'move_ply', 'move', 'cp', 'cp_rel', 'cp_loss', 'is_blunder_cp', 'winrate', 'winrate_elo', 'winrate_loss', 'is_blunder_wr', 'opp_winrate', 'white_active', 'active_elo', 'opponent_elo', 'active_won', 'is_capture', 'clock', 'opp_clock', 'clock_percent', 'opp_clock_percent', 'low_time', 'board', 'active_bishop_count', 'active_knight_count', 'active_pawn_count', 'active_queen_count', 'active_rook_count', 'is_check', 'num_legal_moves', 'opp_bishop_count', 'opp_knight_count', 'opp_pawn_count', 'opp_queen_count', 'opp_rook_count']


In [7]:
print(df.shape)

df.head()

(65000, 47)


,game_id,type,result,white_player,black_player,white_elo,black_elo,time_control,num_ply,termination,...,active_pawn_count,active_queen_count,active_rook_count,is_check,num_legal_moves,opp_bishop_count,opp_knight_count,opp_pawn_count,opp_queen_count,opp_rook_count
0,mAEz2AcC,Rapid,0-1,AliKhaled,ifospor,1702,1628,600+0,36,Normal,...,8,2,2,0,20,2,2,8,2,2
1,mAEz2AcC,Rapid,0-1,AliKhaled,ifospor,1702,1628,600+0,36,Normal,...,8,2,2,0,20,2,2,8,2,2
2,mAEz2AcC,Rapid,0-1,AliKhaled,ifospor,1702,1628,600+0,36,Normal,...,8,2,2,0,29,2,2,8,2,2
3,mAEz2AcC,Rapid,0-1,AliKhaled,ifospor,1702,1628,600+0,36,Normal,...,8,2,2,0,29,2,2,8,2,2
4,mAEz2AcC,Rapid,0-1,AliKhaled,ifospor,1702,1628,600+0,36,Normal,...,8,2,2,0,27,2,2,8,2,2


In [8]:
data = df.loc[:, ['white_active', 'board', 'cp_rel', 'white_elo', 'black_elo', 'move']]
data.head()

,white_active,board,cp_rel,white_elo,black_elo,move
0,True,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,0.10,1702,1628,e2e4
1,False,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-0.12,1702,1628,e7e5
2,True,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,0.37,1702,1628,g1f3
3,False,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,-0.23,1702,1628,b8c6
4,True,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,0.15,1702,1628,f1c4


In [9]:
from torch.utils.data import IterableDataset

class ChessIterableDataset(IterableDataset): # TODO! Write docstrings 

    def __init__(self, csv_files, chunksize):
        self.csv_files = csv_files
        self.chunksize = chunksize

    def process_chunk(self, chunk):
        # Process the chunk and return a list of samples
        samples = []
        for index, row in chunk.iterrows():
            board = row['board']
            white_to_move = row['white_active']
            cp_rel = row['cp_rel']
            move = row['move']

            # Convert data to tensors
            board_tensor = self.fen_to_tensor(board)
            white_to_move = torch.tensor(white_to_move, dtype=torch.float32)
            cp_rel = torch.tensor(cp_rel, dtype=torch.float32)
            move = self.move_to_tensor(move)

            samples.append({'fen': board_tensor, 'fen_str': board, 'white_to_move': white_to_move, 'cp_rel': cp_rel, 'move': move})
        return samples


    def __getitem__(self, idx):
        board = self.dataframe.iloc[idx]['board']
        white_to_move = self.dataframe.iloc[idx]['white_active']
        cp_rel = self.dataframe.iloc[idx]['cp_rel']
        move = self.dataframe.iloc[idx]['move']

        # Convert data to tensors
        board_tensor = self.fen_to_tensor(board)
        white_to_move = torch.tensor(white_to_move, dtype=torch.float32)
        cp_rel = torch.tensor(cp_rel, dtype=torch.float32)
        move = self.move_to_tensor(move)

        return {'fen': board_tensor, 'fen_str': board, 'white_to_move': white_to_move, 'cp_rel': cp_rel, 'move': move}


    def __iter__(self):
        for csv_file in self.csv_files:
            chunk_iter = pd.read_csv(csv_file, chunksize=self.chunksize)
            for chunk in chunk_iter:
                yield from self.process_chunk(chunk) # Returns a list of yielded elements
    
    def move_to_tensor(self, move):
        # Convert the move to a source and destination square
        source_square = move[:2]
        dest_square = move[2:]

        # Convert the squares to indices
        source_index = self.square_to_index(source_square)
        dest_index = self.square_to_index(dest_square)

        move_tensor = torch.zeros(64, 64)
        move_tensor[source_index][dest_index] = 1

        return move_tensor

    def square_to_index(self, square):
        rank = 8 - int(square[1])  # Ranks numbered 8 to 1
        file = ord(square[0]) - ord('a')  # Files lettered a to h
        return rank * 8 + file

    def output_to_move(self, output):
        output_matrix = output.view(64, 64)

        # Find indices of maximum val in the output matrix
        source_index, dest_index = torch.unravel_index(torch.argmax(output_matrix), output_matrix.shape)

        # Convert the indices to a move in algebraic notation
        move = self.index_to_square(source_index.item()) + self.index_to_square(dest_index.item())

        return move


    def modify_output(self, output):
        # Add a small random noise to the output tensor
        noise = torch.randn_like(output) * 0.01
        output += noise

        return output

    def index_to_square(self, index):
        # Convert the index to a rank and file
        rank = 8 - (index // 8)  # Ranks are numbered from 8 to 1
        file = chr((index % 8) + ord('a'))  # Files are lettered from a to h

        # Return the square in algebraic notation
        return file + str(rank)

    def fen_to_tensor(self, fen):
        # Define a mapping from pieces to integers
        piece_to_int = {
            'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
            'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6,
            '.': 0
        }

        # Split the FEN string into parts
        parts = fen.split(' ')
        ranks = parts[0].split('/') # Only process the board position (the first part)

        # Convert the ranks to a list of integers
        board = []
        for rank in ranks:
            for char in rank:
                if char.isdigit():
                    # If the character is a digit, add that many zeros to the board
                    board.extend([0] * int(char))
                else:
                    # Otherwise, add the integer representation of the piece to the board
                    board.append(piece_to_int[char])

        # Convert the board to a tensor
        board_tensor = torch.tensor(board, dtype=torch.float32)

        return board_tensor#.view(-1)



In [10]:
import torch.nn as nn
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64*64, 1024)  # 64*64 input nodes, 256 output nodes
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 64*64)  # 256 input nodes, 64*64 output nodes

    def forward(self, x):
        x = torch.flatten(x)#x.view(x.size(0), -1)  # Flatten the input tensors
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
def is_legal_move(fen, move):
    if move[:2] == move[2:]:
        return False

    # Create a chess board from the FEN string
    board = chess.Board(fen)

    # Convert the move to a chess.Move object
    move = chess.Move.from_uci(move)

    # Check if the move is legal
    return move in board.legal_moves


def custom_loss(dataset, output, target, fen, illegal_move_penalty=1000.0):
    # Calculate the MSE loss
    output = output.view(-1, 64, 64)
    mse_loss = nn.MSELoss()(output, target)

    # Add a penalty for illegal moves
    for i in range(output.shape[0]):
        move = dataset.output_to_move(output[i])
        if not is_legal_move(fen[i], move):
            mse_loss += illegal_move_penalty

    return mse_loss

In [15]:
def train(model, dataset, data_loader, criterion, optimizer, num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        try:
            for i, data in enumerate(data_loader):
            
                # Get the inputs and labels
                inputs = data['fen'].to(device)
                labels = data['move'].to(device)
                fen = data['fen_str']

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(inputs)
                loss = criterion(dataset, outputs, labels, fen)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

        except KeyboardInterrupt:
            print("Finished Training Early!")
            break
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(data_loader)}')


In [16]:
from torch.utils.data import DataLoader
import glob

# Get a list of all CSV files in the directory
csv_files = glob.glob(f'{path}/*')

# Create a dataset
dataset = ChessIterableDataset(csv_files, chunksize=1000)

# Create a data loader
data_loader = DataLoader(dataset, batch_size=64)

# Create a model
model = Net()
model = model.to(device)

# Define a loss function and optimizer
criterion = custom_loss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
train(model, dataset, data_loader, custom_loss, optimizer, num_epochs)

Finished Training Early!


In [ ]:
def modify_output(output):
    # Add a small random noise to the output tensor
    noise = torch.randn_like(output) * 0.01
    output += noise

    return output


def predict(model, fen):
    # Convert the FEN string to a tensor
    fen_tensor = dataset.fen_to_tensor(fen)

    # Add an extra dimension to the tensor and move it to the same device as the model
    input = fen_tensor.unsqueeze(0).to(next(model.parameters()).device)

    # Set the model to evaluation mode
    model.eval()

    # Get the model's predictions
    with torch.no_grad():
        output = model(input)

    output = output.cpu() # Move tensor back to cpu

    # Convert the output tensor to a move
    move = dataset.output_to_move(output[0])

    # If the move is illegal, modify the output tensor to suggest a different move
    while not is_legal_move(fen, move):
        output = modify_output(output)
        move = dataset.output_to_move(output[0])

    return move


In [ ]:
test_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'  # Starting position
test_move = predict(model, test_fen)
print(f'The predicted move is: {test_move}')


In [ ]:
board = chess.Board()